# Import lib


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from tqdm import tqdm

from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Loading data

In [4]:
df = pd.read_csv('/kaggle/input/airline2/text_cleaning.csv')
df.head()

,Review,Recommended,Overall rating,Review_cleaned,Review_bigrams,Review_trigrams
0,i bought roundtrip and return tickets from _gp...,no,1.0,buy roundtrip return ticket _gpe_ _gpe_ via _g...,buy roundtrip return ticket _gpe_ _gpe_ via _g...,buy roundtrip return ticket _gpe_ _gpe_ via _g...
1,i am shocked at how far aeroflot standards hav...,no,1.0,shock far aeroflot standard fall since invade ...,shock far aeroflot standard fall since invade ...,shock far aeroflot standard fall since invade ...
2,aeroflot has set high standards of achieving a...,no,5.0,aeroflot set high standard achieve star airlin...,aeroflot set high standard achieve star airlin...,aeroflot set high standard achieve star airlin...
3,best airline in _gpe_ very nice staff comforta...,yes,10.0,best airline _gpe_ nice staff comfortable seat...,best airline _gpe_ nice staff comfortable seat...,best airline _gpe_ nice staff comfortable seat...
4,i was very impressed with the staff many thank...,yes,10.0,impress staff many thank aeroflot representati...,impress staff many thank aeroflot representati...,impress staff many thank aeroflot representati...


In [5]:
X = df['Review_cleaned']
y = df['Overall rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize

In [6]:
tokenizedText_train=[nltk.word_tokenize(item) for item in tqdm(X_train, total=len(X_train))]
tokenizedText_test=[nltk.word_tokenize(item) for item in tqdm(X_test, total=len(X_test))]

100%|██████████| 13694/13694 [00:12<00:00, 1131.04it/s]


# GloVe

In [7]:
glove_path = "/kaggle/input/glove6b300dtxt/glove.6B.300d.txt"

In [8]:
def load_glove_model(glove_file):
    print("Loading Glove Model")
    f = open(glove_file, 'r')
    total_lines = sum(1 for line in open(glove_file, 'r'))
    model = {}
    vector_size = 300
    for line in tqdm(f, total=total_lines, desc="Processing GloVe Embeddings"):
        split_line = line.split()
        word = " ".join(split_line[0:len(split_line) - vector_size])
        embedding = np.array([float(val) for val in split_line[-vector_size:]])
        model[word] = embedding
    print("Done.\n" + str(len(model)) + " words loaded!")
    return model

In [9]:
glove = load_glove_model(glove_path)

Loading Glove Model


Processing GloVe Embeddings: 100%|██████████| 400000/400000 [00:49<00:00, 8028.91it/s]

Done.
400000 words loaded!


In [10]:
def sentence_features_v2(s, embeddings=glove,emb_size=300):
    words=[w for w in s if w.isalpha() and w in embeddings]
    if len(words)==0:
        return np.hstack([np.zeros(emb_size)])
    M=np.array([embeddings[w] for w in words])
    return M.mean(axis=0)

In [11]:
X_train = np.array([sentence_features_v2(x) for x in tokenizedText_train])
X_test = np.array([sentence_features_v2(x) for x in tokenizedText_test])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((54773, 300), (13694, 300), (54773,), (13694,))

# ML models

## SVM

In [12]:
clf = LinearSVC(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.62      0.98      0.76      5602
         2.0       0.16      0.02      0.04      1476
         3.0       0.15      0.02      0.03       897
         4.0       0.00      0.00      0.00       519
         5.0       0.20      0.00      0.01       524
         6.0       0.20      0.00      0.00       418
         7.0       0.18      0.06      0.09       614
         8.0       0.28      0.29      0.29      1029
         9.0       0.29      0.25      0.27      1175
        10.0       0.46      0.70      0.55      1440

    accuracy                           0.52     13694
   macro avg       0.25      0.23      0.20     13694
weighted avg       0.40      0.52      0.42     13694



## Logistic Regression

In [13]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.64      0.97      0.77      5602
         2.0       0.22      0.03      0.06      1476
         3.0       0.15      0.04      0.07       897
         4.0       0.12      0.01      0.02       519
         5.0       0.18      0.06      0.08       524
         6.0       0.16      0.04      0.07       418
         7.0       0.21      0.15      0.17       614
         8.0       0.28      0.32      0.30      1029
         9.0       0.33      0.29      0.31      1175
        10.0       0.49      0.63      0.56      1440

    accuracy                           0.53     13694
   macro avg       0.28      0.25      0.24     13694
weighted avg       0.42      0.53      0.45     13694



## Random Forest

In [14]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.57      0.98      0.72      5602
         2.0       0.16      0.01      0.02      1476
         3.0       0.11      0.01      0.01       897
         4.0       0.14      0.00      0.00       519
         5.0       0.15      0.00      0.01       524
         6.0       0.12      0.00      0.01       418
         7.0       0.19      0.04      0.06       614
         8.0       0.23      0.18      0.20      1029
         9.0       0.26      0.28      0.27      1175
        10.0       0.42      0.50      0.46      1440

    accuracy                           0.50     13694
   macro avg       0.24      0.20      0.18     13694
weighted avg       0.37      0.50      0.39     13694



## Decision Tree

In [15]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.64      0.62      0.63      5602
         2.0       0.14      0.16      0.15      1476
         3.0       0.10      0.10      0.10       897
         4.0       0.07      0.07      0.07       519
         5.0       0.07      0.06      0.06       524
         6.0       0.04      0.04      0.04       418
         7.0       0.10      0.12      0.11       614
         8.0       0.17      0.17      0.17      1029
         9.0       0.21      0.21      0.21      1175
        10.0       0.28      0.26      0.27      1440

    accuracy                           0.35     13694
   macro avg       0.18      0.18      0.18     13694
weighted avg       0.35      0.35      0.35     13694



##Gaussian NB

In [16]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.71      0.74      0.72      5602
         2.0       0.17      0.12      0.14      1476
         3.0       0.12      0.07      0.09       897
         4.0       0.08      0.05      0.06       519
         5.0       0.08      0.08      0.08       524
         6.0       0.09      0.37      0.15       418
         7.0       0.14      0.18      0.16       614
         8.0       0.25      0.15      0.19      1029
         9.0       0.29      0.20      0.24      1175
        10.0       0.38      0.38      0.38      1440

    accuracy                           0.41     13694
   macro avg       0.23      0.23      0.22     13694
weighted avg       0.41      0.41      0.41     13694



## KNN

In [17]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.61      0.91      0.73      5602
         2.0       0.14      0.08      0.10      1476
         3.0       0.12      0.06      0.08       897
         4.0       0.08      0.03      0.05       519
         5.0       0.09      0.03      0.05       524
         6.0       0.05      0.03      0.03       418
         7.0       0.13      0.14      0.13       614
         8.0       0.21      0.22      0.22      1029
         9.0       0.27      0.21      0.24      1175
        10.0       0.47      0.25      0.33      1440

    accuracy                           0.45     13694
   macro avg       0.22      0.20      0.20     13694
weighted avg       0.38      0.45      0.40     13694



## AdaBoost

In [18]:
adb = AdaBoostClassifier()
adb.fit(X_train, y_train)
y_pred = adb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.61      0.95      0.74      5602
         2.0       0.19      0.02      0.03      1476
         3.0       0.14      0.01      0.03       897
         4.0       0.12      0.00      0.00       519
         5.0       0.17      0.01      0.01       524
         6.0       0.16      0.03      0.05       418
         7.0       0.18      0.15      0.16       614
         8.0       0.24      0.24      0.24      1029
         9.0       0.29      0.25      0.27      1175
        10.0       0.39      0.54      0.45      1440

    accuracy                           0.50     13694
   macro avg       0.25      0.22      0.20     13694
weighted avg       0.39      0.50      0.41     13694

